<a href="https://colab.research.google.com/github/mhsingell/BERTY/blob/main/BERT_Model_Accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#My Package
import BERT_Project_Package as bpp

#General
import numpy as np
import pandas as pd

#For all your pickle needs
import pickle

#For Train and Test Split
import random

#For Running BERT
from transformers import BertTokenizer, BertForPreTraining, BertModel, TFBertForMaskedLM, BertConfig, BertForMaskedLM
from transformers import BertTokenizer, LineByLineTextDataset, BertModel, BertConfig, BertForMaskedLM, DataCollatorForLanguageModeling
import torch

#For metrics
from datasets import load_metric

#for graphing
import seaborn as sns

ModuleNotFoundError: ignored

In [ ]:
#Load models
model_pt = torch.load('model_pt11.pth', map_location=torch.device('cpu'))
model_ft = torch.load('model_ft6.pth', map_location=torch.device('cpu'))
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertForMaskedLM.from_pretrained('bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import json
with open('easier.json') as d:
    x_dict = json.load(d)

In [ ]:
#Load Pageview Data
with open('wiki_small.txt') as f:
    lines = f.readlines()

In [ ]:
#Attach pageviews to data dictionary
x_dict_pvs = bpp.attach_pvs(lines, x_dict)

In [ ]:
train, validate, test = bpp.trainvaltest(x_dict_pvs)

In [ ]:
acc_vb, pred_vb, df_vb = bpp.find_accuracy(validate, model)

In [ ]:
acc_ptb, pred_ptb, df_ptb = bpp.find_accuracy(validate, model_pt)

In [ ]:
acc_ftb, pred_ftb, df_ftb = bpp.find_accuracy(validate, model_ft)

In [ ]:
#Graph accuracy by pageview decile
x_dict_pvs_df = pd.DataFrame(x_dict_pvs)
x_dict_pvs_df['dec_pv'] = pd.qcut(x_dict_pvs_df["pagevs"], 10,
                               labels = False)
validate_titles = validate["title"]
val_df = x_dict_pvs_df.loc[x_dict_pvs_df["title"].isin(validate_titles)]
val_df = val_df.set_index('title')

In [ ]:
#BERT Vanilla
for_graph_df_vb = df_vb
for_graph_df_vb["title"] = validate["title"]
for_graph_df_vb = for_graph_df_vb.set_index('title')

graph_val_vb = pd.merge(val_df, for_graph_df_vb, left_index=True, right_index=True, how='outer')

result1 = graph_val_vb.groupby('dec_pv', as_index=False)['correct'].mean()
result1.sort_values('dec_pv', ascending=False)


In [ ]:
sns.lineplot(data=result1, x="dec_pv", y="correct")

In [ ]:
#BERT Vanilla, probability of accuracy 
result_p = graph_val_vb.groupby('dec_pv', as_index=False)['prob_score'].mean()
result_p.sort_values('dec_pv', ascending=False)


In [ ]:
sns.lineplot(data=result_p, x="dec_pv", y="prob_score")

In [ ]:
#Pre-trained BERT
for_graph_df_ptb = df_ptb
for_graph_df_ptb["title"] = validate["title"]
for_graph_df_ptb = for_graph_df_ptb.set_index('title')

graph_val_ptb = pd.merge(val_df, for_graph_df_ptb, left_index=True, right_index=True, how='outer')

result2 = graph_val_ptb.groupby('dec_pv', as_index=False)['correct'].mean()
result2.sort_values('dec_pv', ascending=False)


In [ ]:
sns.lineplot(data=result2, x="dec_pv", y="correct")

In [ ]:
#Pre-trained BERT, probability of accuracy 
result_p_ptb = graph_val_ptb.groupby('dec_pv', as_index=False)['prob_score'].mean()
result_p_ptb.sort_values('dec_pv', ascending=False)


In [ ]:
sns.lineplot(data=result_p_ptb, x="dec_pv", y="prob_score")

In [ ]:
#Fine-tuned BERT
for_graph_df_ftb = df_ftb
for_graph_df_ftb["title"] = validate["title"]
for_graph_df_ftb = for_graph_df_ftb.set_index('title')

graph_val_ftb = pd.merge(val_df, for_graph_df_ftb, left_index=True, right_index=True, how='outer')

result3 = graph_val_ftb.groupby('dec_pv', as_index=False)['correct'].mean()
result3.sort_values('dec_pv', ascending=False)


In [ ]:
sns.lineplot(data=result3, x="dec_pv", y="correct")

In [ ]:
#Fine-tuned BERT, probability of accuracy 
result_p_ftb = graph_val_ftb.groupby('dec_pv', as_index=False)['prob_score'].mean()
result_p_ftb.sort_values('dec_pv', ascending=False)


In [ ]:
sns.lineplot(data=result_p_ftb, x="dec_pv", y="prob_score")

In [ ]:
#Can we say something about what BERT is picking up from the training data? 

In [ ]:
#Small friendly train dataset
small_index = list(range(0,10))

small = {"id": [], "url": [], "title": [], "text": [], "pagevs": []}

for indx in range(len(small_index)):
    myidx = small_index[indx]
    for key in train.keys():
        my_key = train[key]
        small[key].append(train[key][myidx])

In [ ]:
acc_vb_t, pred_vb_t, df_vb_t = bpp.find_accuracy(train, model)

KeyboardInterrupt: ignored

In [ ]:
acc_vb_t

In [ ]:
pred_vb_t

In [ ]:
acc_ptb_t, pred_ptb_t, df_ptb_t = bpp.find_accuracy(train, model_pt)

In [ ]:
acc_ptb_t

In [ ]:
pred_ptb_t

In [ ]:
acc_ftb_t, pred_ftb_t, df_ftb_t = bpp.find_accuracy(validate, model_ft)

In [ ]:
acc_ftb_t

In [ ]:
pred_ftb_t

In [ ]:
#Graph accuracy by pageview decile, train
x_dict_pvs_df = pd.DataFrame(x_dict_pvs)
x_dict_pvs_df['dec_pv'] = pd.qcut(x_dict_pvs_df["pagevs"], 10,
                               labels = False)
train_titles = train["title"]
trn_df = x_dict_pvs_df.loc[x_dict_pvs_df["title"].isin(train_titles)]
trn_df = trn_df.set_index('title')

In [ ]:
#BERT Vanilla
for_graph_df_vb_t = df_vb_t
for_graph_df_vb_t["title"] = train["title"]
for_graph_df_vb_t = for_graph_df_vb_t.set_index('title')

graph_val_vb_t = pd.merge(trn_df, for_graph_df_vb_t, left_index=True, right_index=True, how='outer')

result1_t = graph_val_vb_t.groupby('dec_pv', as_index=False)['correct'].mean()
result1_t.sort_values('dec_pv', ascending=False)


In [ ]:
sns.lineplot(data=result1_t, x="dec_pv", y="correct")

In [ ]:
##Pre-trained BERT
for_graph_df_ptb_t = df_ptb_t
for_graph_df_ptb_t["title"] = train["title"]
for_graph_df_ptb_t = for_graph_df_ptb_t.set_index('title')

graph_val_ptb_t = pd.merge(trn_df, for_graph_df_ptb_t, left_index=True, right_index=True, how='outer')

result1_ptb_t = graph_val_ptb_t.groupby('dec_pv', as_index=False)['correct'].mean()
result1_ptb_t.sort_values('dec_pv', ascending=False)



In [ ]:
sns.lineplot(data=result1_ptb_t, x="dec_pv", y="correct")

In [ ]:
##Fine_tuned BERT
for_graph_df_ft_t = df_ftb_t
for_graph_df_ft_t["title"] = train["title"]
for_graph_df_ft_t = for_graph_df_ft_t.set_index('title')

graph_val_ft_t = pd.merge(trn_df, for_graph_df_ft_t, left_index=True, right_index=True, how='outer')

result1_ft_t = graph_val_ft_t.groupby('dec_pv', as_index=False)['correct'].mean()
result1_ft_t.sort_values('dec_pv', ascending=False)



In [ ]:
sns.lineplot(data=result1_ft_t, x="dec_pv", y="correct")